In [ ]:
import datetime
import os
import numpy as np
import pandas as pd
import requests
import smtplib
from bs4 import BeautifulSoup
from email.message import EmailMessage

In [ ]:
# Declare Variables
cname = "Assam"

date = "02/05/2023"
mobilink_date = date

if date == "":
    date = datetime.datetime.now().strftime("%d/%m/%Y")

if mobilink_date == "":
    mobilink_date = datetime.datetime.now().strftime("%d/%m/%Y")

sender_email = "techassam@gmail.com"
sender_password = ""
recepients = [
    # "dodarrang.as@indiapost.gov.in",
    # "docachar.as@indiapost.gov.in",
    # "donalbari.as@indiapost.gov.in",
    # "dotinsukia.as@indiapost.gov.in",
    # "doguwahati.as@indiapost.gov.in",
    # "dogoalpara.as@indiapost.gov.in",
    # "dodibrugarh.as@indiapost.gov.in",
    # "dosibsagar.as@indiapost.gov.in",
    # "donagaon.as@indiapost.gov.in"
    # 'eliezer.kt@gmail.com',
    # 'tek2991@gmail.com',
]

reports = {
    '1' : 'Details of Post Offices having daily synch failure for more than 48 hours',
    '2' : 'Details of Post Offices not performded EOD in DPMS',
    '3' : 'Details of Bos for which  BO slips are not generated in CSI',
    '4' : 'Details of Post offices where both NSP 1 & 2 are down',
    '23' : 'Details of Post offices where both NSP 1 & 2 are down but ticket is not raised',
    '5' : 'Details of BOs which have not logged in Darpan during Office Hours(9AM - 2PM)'
}

divisions = {
    'Cachar Division' : ['Cachar'],
    'Darrang Division' : ['Darrang'],
    'Dibrugarh Division' : ['Dibrugarh'],
    'Goalpara Division' : ['Goalpara'],
    'Guwahati Division' : ['Guwahati', 'Guahati'],
    'Nagaon Division' : ['Nagaon'],
    'Nalbari Division' : ['Nalbari'],
    'Sibsagar Division' : ['Sibsagar', 'Sivasagar', 'Sivsagar', 'Shivsagar'],
    'Tinsukia Division' : ['Tinsukia']
}

# Check if logs csv exists
if not os.path.exists("logs.csv"):
    df = pd.DataFrame(columns=["date", "report_id", "report_name", "status", "log_date"])
    df.to_csv("logs.csv", index=False)

# url encoding
rpdate = date.replace("/", "%2f")

base_url = "https://mis.cept.gov.in/viewDetails.aspx"

url = base_url + "?cname=" + cname + "&rpdate=" + rpdate
webpage = requests.get(url)

# Declare form id
form_id = "form1"

# Find form
form = BeautifulSoup(webpage.text, "html.parser").find("form", {"id": form_id})

# get the inputs
inputs = form.find_all("input")

# create a dictionary of the inputs
input_dict = {}
accepted_inputs = ["__VIEWSTATE", "__VIEWSTATEGENERATOR", "__EVENTVALIDATION"]
for i in inputs:
    if i.has_attr("value") and i["name"] in accepted_inputs:
        input_dict[i["name"]] = i["value"]

# Create headers
headers = {
    "authority": "mis.cept.gov.in",
    "method": "POST",
    "path": "/viewDetails.aspx?cname=" + cname + "&rpdate=" + rpdate,
    "scheme": "https"
}

# Create a folder by the date inside data folder
if not os.path.exists("data/" + date.replace("/", "-")):
    os.makedirs("data/" + date.replace("/", "-"))

# check if consolidated excel file exists for the date
if not os.path.exists("data/" + date.replace("/", "-") + "/consolidated.xlsx"):
    # Create a consolidated excel file
    consolidated_df = pd.DataFrame(columns=["Division Name", "POS High Sync Defaulter dated " + mobilink_date, "POS Daily Sync Defaulter dated " + mobilink_date ])

    # Add report names as column names
    for key, value in reports.items():
        # Remove "Details of " from the report name and suffix it with date
        val = value.replace("Details of ", "") + " dated: " + date
        consolidated_df[val] = ""

    #  Add division names as rows using concat
    for key, value in divisions.items():
        consolidated_df = pd.concat([consolidated_df, pd.DataFrame([[key]], columns=['Division Name'])])

    # Save the file
    consolidated_df.to_excel("data/" + date.replace("/", "-") + "/consolidated.xlsx", index=False)
    

# Loop through the reports
for key, value in reports.items():

    # Check if the report has already been sent
    logs_df = pd.read_csv("logs.csv")
    if logs_df[(logs_df["date"] == date) & (logs_df["report_id"] == int(key))].shape[0] > 0:
        # enter in the logs
        # logs_df = logs_df.append({
        #     "date": date,
        #     "report_id": key,
        #     "report_name": value,
        #     "status": "skipped",
        #     "log_date": datetime.datetime.now().strftime("%d/%m/%Y %H:%M:%S")
        # }, ignore_index=True)
        # Append method has been deprecated, use concat instead
        logs_df = pd.concat([logs_df, pd.DataFrame([[date, key, value, "skipped", datetime.datetime.now().strftime("%d/%m/%Y %H:%M:%S")]], columns=["date", "report_id", "report_name", "status", "log_date"])])
        logs_df.to_csv("logs.csv", index=False)
        # skip the report
        continue

    # Create a dictionary of the fields
    fields = {
        "ctl00$ContentPlaceHolder1$DdoTransaction": key,
        "ctl00$ContentPlaceHolder1$DdoComplaint" : cname,
        "ctl00$ContentPlaceHolder1$TxtReportDate" : date,
        "ctl00$ContentPlaceHolder1$btnDelete" : "View Data"
    }

    # Combine the inputs and fields
    form_data = {**input_dict, **fields}

    # Send the request
    response = requests.post(url, data=form_data, headers=headers)

    # Get the table
    table = BeautifulSoup(response.text, "html.parser").find("table", {"id": "ContentPlaceHolder1_gvAll"})

    # Check if the table is empty
    if table is None:
        # Create text file with the report name
        with open("data/" + date.replace("/", "-") + "/" + value + "-" + date.replace("/", "-") + ".txt", "w") as f:
            f.write("No Data")
        continue

    # Convert the table to a dataframe
    df = pd.read_html(str(table))[0]

    # Convert the dataframe to a excel file
    df.to_excel("data/" + date.replace("/", "-") + "/" + value + " " + date.replace("/", "-") + ".xlsx", index=False)

    # Field name to group by
    field_name = "DIVISIONNAME"

    # Group by the field name
    grouped = df.groupby(field_name).count()['CIRCLENAME']
    grouped.loc['Total'] = grouped.sum()

    # Rename the column CIRCLENAME to OFFICES
    grouped.rename("OFFICES", inplace=True)

    # Save the grouped data to an excel file
    grouped.to_excel("data/" + date.replace("/", "-") + "/" + value + " " + date.replace("/", "-") + " grouped.xlsx")
    
    consolidated_df = pd.read_excel("data/" + date.replace("/", "-") + "/consolidated.xlsx")

    # Loop throught he grouped rows
    for index, row in grouped.items():
        # first word of the index is the division name
        div_name = index.split(" ")[0]

        name = ''

        # Search the div_name in the lists of divisions 
        for k, v in divisions.items():
            for div in v:
                # case insensitive search
                if div.lower() == div_name.lower():
                    name = k
                    break

        # Remove "Details of " from the report name and suffix it with date
        val = value.replace("Details of ", "") + " dated: " + date
        
        # Update the consolidated_df where the division name is like the name and the column name is like the value
        consolidated_df.loc[consolidated_df["Division Name"] == name, val] = row

        # Add total row
        # consolidated_df.loc[consolidated_df["Division Name"].str.contains("Total", case=False), value] = grouped.sum()
        # Save the file
        consolidated_df.to_excel("data/" + date.replace("/", "-") + "/consolidated.xlsx", index=False)

    # Add the log to the csv
    logs_df = pd.read_csv("logs.csv")
    logs_df = pd.concat([logs_df, pd.DataFrame([[date, key, value, "success", datetime.datetime.now().strftime("%d/%m/%Y %H:%M:%S")]], columns=logs_df.columns)])
    logs_df.to_csv("logs.csv", index=False)

In [ ]:
# Check for csv file with the date in mobilink folder
if not os.path.exists("mobilink/" + mobilink_date.replace("/", "-") + ".csv"):
    print("Mobilink file does not exist")
else:
    print("Mobilink file exists")
    mobilink_df = pd.read_csv("mobilink/" + mobilink_date.replace("/", "-") + ".csv")
    # Trim the spaces from the columns
    mobilink_df["CIRCLE"] = mobilink_df["CIRCLE"].str.strip()
    mobilink_df["HIGH_SYNC_STATUS"] = mobilink_df["HIGH_SYNC_STATUS"].str.strip()
    mobilink_df["DAILY_SYNC_STATUS"] = mobilink_df["DAILY_SYNC_STATUS"].str.strip()
    mobilink_df["DIVISION"] = mobilink_df["DIVISION"].str.strip()


    # Remove data where the CIRCLE is not "Assam Circle"
    mobilink_df = mobilink_df[mobilink_df["CIRCLE"] == "Assam Circle"]
    # Remove data where the HIGH_SYNC_STATUS & DAILY_SYNC_STATUS is REGULAR
    mobilink_df = mobilink_df[(mobilink_df["HIGH_SYNC_STATUS"] == "DEFAULTER") | (mobilink_df["DAILY_SYNC_STATUS"] == "DEFAULTER")]

    # Replace the csv file with the new data
    mobilink_df.to_csv("mobilink/" + mobilink_date.replace("/", "-") + ".csv", index=False)

    high_df = mobilink_df[mobilink_df["HIGH_SYNC_STATUS"] == "DEFAULTER"]
    daily_df = mobilink_df[mobilink_df["DAILY_SYNC_STATUS"] == "DEFAULTER"]

    grouped_high = high_df.groupby("DIVISION").count()["CIRCLE"]
    grouped_daily = daily_df.groupby("DIVISION").count()["CIRCLE"]

    consolidated_df = pd.read_excel("data/" + date.replace("/", "-") + "/consolidated.xlsx")

    for index, row in grouped_high.items():
        div_name = index.split(" ")[0]
        name = ''
        # Search the div_name in the lists of divisions 
        for k, v in divisions.items():
            for div in v:
                # case insensitive search
                if div.lower() == div_name.lower():
                    name = k
                    break
                
        consolidated_df.loc[consolidated_df["Division Name"] == name, "POS High Sync Defaulter dated " + mobilink_date] = row


    for index, row in grouped_daily.items():
        div_name = index.split(" ")[0]
        name = ''
        # Search the div_name in the lists of divisions 
        for k, v in divisions.items():
            for div in v:
                # case insensitive search
                if div.lower() == div_name.lower():
                    name = k
                    break
                
        consolidated_df.loc[consolidated_df["Division Name"] == name, "POS Daily Sync Defaulter dated " + mobilink_date] = row

    # Replace NaN with 0
    consolidated_df = consolidated_df.fillna(0)

    # Index starts from 1 instead of 0
    consolidated_df.index += 1


    # Add a total row
    consolidated_df.loc["Total"] = consolidated_df.sum()

    # Change Division Name to of the last row to empty
    consolidated_df["Division Name"].iloc[-1] = ""

    consolidated_df.to_excel("data/" + date.replace("/", "-") + "/consolidated.xlsx")
    
# consolidated_df

In [ ]:
with pd.ExcelWriter("Reports/" + date.replace("/", "-") + "_consolidated_data" + ".xlsx",) as writer:
# Loop through the reports
    for key, value in reports.items():
        # load the excel file
        xyz_df = pd.read_excel("data/" + date.replace("/", "-") + "/" + value + " " + date.replace("/", "-") + ".xlsx")
        # save the file
        xyz_df.to_excel(writer, sheet_name=value[11:], index=False)

In [ ]:
# Convert the consolidated_df to a html table
html = consolidated_df.to_html(justify="justify-all", classes="table table-bordered table-striped table-hover").replace('<tr>', '<tr align="center">').replace('.0</td>', '</td>').replace('<td>0</td>', '<td style="background:lightgreen">0</td>')

# print(html)
